<a href="https://colab.research.google.com/github/VitalyGladyshev/gb_nlp/blob/main/HW_07_NLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ДЗ 07 NLP

## Задание 1

Переделать генерацию текста по символам в генерацию по словам

In [1]:
import numpy as np
import tensorflow as tf
import string
from tqdm import tqdm
import os

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [3]:
%cd "./gdrive/My Drive/Colab Notebooks/NLP/"

/content/gdrive/My Drive/Colab Notebooks/NLP


In [4]:
path_to_file = 'evgenyi_onegin.txt'

# text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
# # length of text is the number of characters in it
# print('Length of text: {} characters'.format(len(text)))

vocab = set()
vocab.add("\n")
vocab.add(".")
vocab.add(",")
with open(path_to_file, "r", encoding='utf-8', errors='ignore') as f:
    for line in tqdm(f):
        line = ''.join([ch for ch in list(line) if ch not in string.punctuation])
        words = line.lower().strip().split(" ")
        for word in words:
            vocab.add(word)

6505it [00:00, 23597.72it/s]


In [5]:
"\n" in vocab

True

In [6]:
word2vec = {}
idx2word = np.array(list(vocab))

for i, word in enumerate(vocab):
    word2vec[word] = i

In [7]:
word2vec["\n"], word2vec["."], idx2word[word2vec["."]]

(6975, 1710, '.')

In [8]:
text_as_int = []
with open(path_to_file, "r", encoding='utf-8', errors='ignore') as f:
    for line in tqdm(f):
        words = line.lower().strip().split(" ")
        for word in words:
            point, comma = False, False
            if "," in word:
                comma = True
            if "." in word:
                point = True
            word = ''.join([ch for ch in list(word) if ch not in string.punctuation])
            text_as_int.append(word2vec[word])
            if comma:
                text_as_int.append(word2vec[","])
            if point:
                text_as_int.append(word2vec["."])
        text_as_int.append(word2vec["\n"])

6505it [00:00, 109866.62it/s]


In [9]:
text_as_int[:15]

[2537,
 3652,
 6890,
 6975,
 0,
 6975,
 3934,
 2717,
 6975,
 6210,
 2768,
 4618,
 6975,
 0,
 6975]

In [10]:
len(text_as_int)

35895

### train and target

In [11]:
# The maximum length sentence you want for a single input in characters
seq_length = 100
examples_per_epoch = len(text_as_int)//(seq_length+1)

# Create training examples / targets
word_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

for i in word_dataset.take(25):
    print(idx2word[i.numpy()])

александр
сергеевич
пушкин





евгений
онегин


роман
в
стихах





не
мысля
гордый
свет
забавить
,


вниманье
дружбы
возлюбя


In [12]:
sequences = word_dataset.batch(seq_length+1, drop_remainder=True)
sequences

<BatchDataset shapes: (101,), types: tf.int32>

In [13]:
for item in sequences.take(3):
    # print(idx2word[item.numpy().astype(int)])
    print(' '.join(idx2word[item.numpy()]))

александр сергеевич пушкин 
  
 евгений онегин 
 роман в стихах 
  
 не мысля гордый свет забавить , 
 вниманье дружбы возлюбя , 
 хотел бы я тебе представить 
 залог достойнее тебя , 
 достойнее души прекрасной , 
 святой исполненной мечты , 
 поэзии живой и ясной , 
 высоких дум и простоты 
 но так и быть  рукой пристрастной 
 прими собранье пестрых глав , 
 полусмешных , полупечальных , 
 простонародных , идеальных , 
 небрежный плод моих забав , 
 бессонниц , легких вдохновений , 
 незрелых и увядших лет , 

ума холодных наблюдений 
 и сердца горестных замет . 
  
 глава первая 
  
 и жить торопится и чувствовать спешит . 
 кн . вяземский . 
  
 i 
  
 мой дядя самых честных правил , 
 когда не в шутку занемог , 
 он уважать себя заставил 
 и лучше выдумать не мог . 
 его пример другим наука 
 но , боже мой , какая скука 
 с больным сидеть и день и ночь , 
 не отходя ни шагу прочь 
 какое низкое коварство 
 полуживого забавлять , 
 ему подушки поправлять
, 
 печально подносить лека

In [14]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

dataset = sequences.map(split_input_target)

In [15]:
for input_example, target_example in  dataset.take(1):
    print('Input data: ', repr(' '.join(idx2word[input_example.numpy()])))
    print('Target data:', repr(' '.join(idx2word[target_example.numpy()])))

Input data:  'александр сергеевич пушкин \n  \n евгений онегин \n роман в стихах \n  \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты \n но так и быть  рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \n небрежный плод моих забав , \n бессонниц , легких вдохновений , \n незрелых и увядших лет ,'
Target data: 'сергеевич пушкин \n  \n евгений онегин \n роман в стихах \n  \n не мысля гордый свет забавить , \n вниманье дружбы возлюбя , \n хотел бы я тебе представить \n залог достойнее тебя , \n достойнее души прекрасной , \n святой исполненной мечты , \n поэзии живой и ясной , \n высоких дум и простоты \n но так и быть  рукой пристрастной \n прими собранье пестрых глав , \n полусмешных , полупечальных , \n простонародных , идеальных , \n неб

In [16]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

dataset

<BatchDataset shapes: ((64, 100), (64, 100)), types: (tf.int32, tf.int32)>

In [17]:
# Length of the vocabulary in chars
vocab_size = len(vocab)

# The embedding dimension
embedding_dim = 256

# Number of RNN units
rnn_units = 1024

In [18]:
def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                                  batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                            return_sequences=True,
                            stateful=True,
                            recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

In [19]:
model = build_model(
            vocab_size=len(vocab),
            embedding_dim=embedding_dim,
            rnn_units=rnn_units,
            batch_size=BATCH_SIZE)

In [20]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(example_batch_predictions.shape, "# (batch_size, sequence_length, vocab_size)")

(64, 100, 8566) # (batch_size, sequence_length, vocab_size)


In [21]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           2192896   
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 8566)          8780150   
Total params: 14,911,350
Trainable params: 14,911,350
Non-trainable params: 0
_________________________________________________________________


In [22]:
sampled_indices = tf.random.categorical(example_batch_predictions[0], num_samples=1)
sampled_indices = tf.squeeze(sampled_indices,axis=-1).numpy()

In [23]:
print("Input: \n", repr(" ".join(idx2word[input_example_batch[0]])))
print()
print("Next Char Predictions: \n", repr(" ".join(idx2word[sampled_indices ])))

Input: 
 'то воля неба я твоя \n вся жизнь моя была залогом \n свиданья верного с тобой \n я знаю , ты мне послан богом , \n до гроба ты хранитель мой . \n ты в сновиденьях мне являлся \n незримый , ты мне был уж мил , \n твой чудный взгляд меня томил , \n в душе твой голос раздавался \n давно . нет , это был не сон \n ты чуть вошел , я вмиг узнала , \n вся обомлела , запылала \n и в мыслях молвила вот он \n не правда ль я тебя слыхала \n ты говорил'

Next Char Predictions: 
 'непонятна печальным забор трубок тетки взбесить маню приехал наши возьми ces послан целовать целя обморок закону медалью прихотливый чуждо общественное рисуя сегодня певец тряслися воротник моим свесть охоты днем олигархических восторгов семьей найдено погрузился сие исполнены беспокоилась утренний способен демоном харитонья волшебному тишине звон уходит невольник усердие езда пустяков писать клоками предузнав модный утреннем видит non румянец невинность чуть эклога слабый хвалить дурака значенье изменять восторж

## Train the model

In [24]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

example_batch_loss = loss(target_example_batch, example_batch_predictions)
print("Prediction shape: ", example_batch_predictions.shape, " # (batch_size, sequence_length, vocab_size)")
print("scalar_loss:      ", example_batch_loss.numpy().mean())

Prediction shape:  (64, 100, 8566)  # (batch_size, sequence_length, vocab_size)
scalar_loss:       9.055292


In [25]:
model.compile(optimizer='adam', loss=loss)

### Configure checkpoints

In [26]:
# Directory where the checkpoints will be saved
checkpoint_dir = './training_checkpoints'
# Name of the checkpoint files
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

### Execute the training

In [27]:
EPOCHS = 200

In [28]:
history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

Epoch 1/200
5/5 [==============================] - 1s 277ms/step - loss: 8.9231
Epoch 2/200
5/5 [==============================] - 2s 352ms/step - loss: 6.4932
Epoch 3/200
5/5 [==============================] - 1s 248ms/step - loss: 6.2706
Epoch 4/200
5/5 [==============================] - 1s 263ms/step - loss: 6.1984
Epoch 5/200
5/5 [==============================] - 1s 276ms/step - loss: 6.1515
Epoch 6/200
5/5 [==============================] - 1s 291ms/step - loss: 6.1051
Epoch 7/200
5/5 [==============================] - 1s 245ms/step - loss: 6.0675
Epoch 8/200
5/5 [==============================] - 1s 260ms/step - loss: 5.9805
Epoch 9/200
5/5 [==============================] - 3s 519ms/step - loss: 5.9293
Epoch 10/200
5/5 [==============================] - 1s 255ms/step - loss: 5.8399
Epoch 11/200
5/5 [==============================] - 1s 255ms/step - loss: 5.7459
Epoch 12/200
5/5 [==============================] - 10s 2s/step - loss: 5.6339
Epoch 13/200
5/5 [=====================

## Generate text

In [29]:
tf.train.latest_checkpoint(checkpoint_dir)

'./training_checkpoints/ckpt_200'

In [30]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)

model.load_weights(tf.train.latest_checkpoint(checkpoint_dir))

model.build(tf.TensorShape([1, None]))

In [31]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (1, None, 256)            2192896   
_________________________________________________________________
gru_1 (GRU)                  (1, None, 1024)           3938304   
_________________________________________________________________
dense_1 (Dense)              (1, None, 8566)           8780150   
Total params: 14,911,350
Trainable params: 14,911,350
Non-trainable params: 0
_________________________________________________________________


In [37]:
def generate_text(model, start_string):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = 500

    # Converting our start string to numbers (vectorizing)
    input_eval = []

    words = start_string.lower().strip().split(" ")
    for word in words:
        point, comma = False, False
        if "," in word:
            comma = True
        if "." in word:
            point = True
        word = ''.join([ch for ch in list(word) if ch not in string.punctuation])
        input_eval.append(word2vec[word])
        if comma:
            input_eval.append(word2vec[","])
        if point:
            input_eval.append(word2vec["."])
    # input_eval = [word2vec[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperature results in more predictable text.
    # Higher temperature results in more surprising text.
    # Experiment to find the best setting.
    temperature = 0.5

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0)
        # using a categorical distribution to predict the character returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1, 0].numpy()

        # Pass the predicted character as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2word[predicted_id])

    return (start_string + ' '.join(text_generated))

In [40]:
start_string = u"Уж небо осенью дышало, \
Уж реже солнышко блистало, \
Короче становился день,"


print(generate_text(model, start_string=start_string))

Уж небо осенью дышало, Уж реже солнышко блистало, Короче становился день,благом плечу брегам плечу плечу удовольствием руке лаковым настоящем барами большом колено ручью впившийся последней удовольствием барами письмах моде почте мере работам солнцем плечу свету видом моде настоящем видом вольному волнами барами плечу последней колено брусничною удовольствием удовольствием колено вами холма мненью барами плечу плечу ляжку ручью брегам плечу плечу плечу ученым конце плечу воспоминаний конце удовольствием тверской моде ним колено родственным моде чинам плечу моде пошлых родственным шумящею плечу плечу темноте удовольствием почте ручью удовольствием окном плечу удовольствием восточной плечу удовольствием удовольствием юностью видом чинам плечу брусничною мере плечу плечу последней плечу родственным плечу наказу барами ручью алтарем дурачки последней конце петербургской шумящею книгах моде окном плечу последней мере почте плечу последней плечу суд последней балтическим волнами моде изумлен

## Задание 2

Запустить seq2seq, seq2seq с внимаием для перевода русских слов + описать наблюдения по качеству